In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os


# Reading Datasets

One class SVM is trained only on normal dataset which will identify the boundraies

In [2]:
#Reading normal dataset
df=pd.read_csv('../input/dataset/InSDN_DatasetCSV/Normal_data.csv')
df.head(5)

,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,185.127.17.56-192.168.20.133-443-53648-6,185.127.17.56,443,192.168.20.133,53648,6,5/2/2020 13:58,245230,44,40,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Normal
1,185.127.17.56-192.168.20.133-443-53650-6,192.168.20.133,53650,185.127.17.56,443,6,5/2/2020 13:58,1605449,107,149,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Normal
2,192.168.20.133-192.168.20.2-35108-53-6,192.168.20.133,35108,192.168.20.2,53,6,5/2/2020 13:58,53078,5,5,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Normal
3,192.168.20.133-192.168.20.2-35108-53-6,192.168.20.2,53,192.168.20.133,35108,6,5/2/2020 13:58,6975,1,1,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Normal
4,154.59.122.74-192.168.20.133-443-60900-6,192.168.20.133,60900,154.59.122.74,443,6,5/2/2020 13:58,190141,13,16,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Normal


In [3]:
#Reading Anomaly dataset for used for testing purpose
df_test=pd.read_csv('../input/dataset/InSDN_DatasetCSV/OVS.csv')
df_test.head(5)

,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,172.17.0.2-200.175.2.130-80-48856-6,200.175.2.130,48856,172.17.0.2,80,6,12/1/2020 1:14,4045,2,5,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BFA
1,172.17.0.2-200.175.2.130-80-48856-6,200.175.2.130,48856,172.17.0.2,80,6,12/1/2020 1:14,284,1,1,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BFA
2,172.17.0.2-200.175.2.130-80-48858-6,200.175.2.130,48858,172.17.0.2,80,6,12/1/2020 1:14,4302,2,5,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BFA
3,172.17.0.2-200.175.2.130-80-48858-6,200.175.2.130,48858,172.17.0.2,80,6,12/1/2020 1:14,83,1,1,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BFA
4,172.17.0.2-200.175.2.130-80-48860-6,200.175.2.130,48860,172.17.0.2,80,6,12/1/2020 1:14,3760,2,5,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BFA


# Data Cleaning

In [4]:
df = df.dropna('columns')# drop columns with NaN
#df = df[[col for col in df if df[col].nunique() > 1]]# keep columns where there are more than 1 unique values
df_test=df_test.dropna('columns')

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.dropna will be keyword-only
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.dropna will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


In [5]:
print(df.dtypes)
print(df.columns)

Flow ID       object
Src IP        object
Src Port       int64
Dst IP        object
Dst Port       int64
              ...   
Idle Mean    float64
Idle Std     float64
Idle Max     float64
Idle Min     float64
Label         object
Length: 84, dtype: object
Index(['Flow ID', 'Src IP', 'Src Port', 'Dst IP', 'Dst Port', 'Protocol',
       'Timestamp', 'Flow Duration', 'Tot Fwd Pkts', 'Tot Bwd Pkts',
       'TotLen Fwd Pkts', 'TotLen Bwd Pkts', 'Fwd Pkt Len Max',
       'Fwd Pkt Len Min', 'Fwd Pkt Len Mean', 'Fwd Pkt Len Std',
       'Bwd Pkt Len Max', 'Bwd Pkt Len Min', 'Bwd Pkt Len Mean',
       'Bwd Pkt Len Std', 'Flow Byts/s', 'Flow Pkts/s', 'Flow IAT Mean',
       'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Tot',
       'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min',
       'Bwd IAT Tot', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max',
       'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags', 'Fwd URG Flags',
       'Bwd URG Flags', 'Fwd Header Len', 'Bwd Header 

In [6]:
df.drop(['Flow ID', 'Src IP' ,'Dst IP' ,'Timestamp','Src Port','Dst Port','Protocol'], inplace=True,axis = 1)
df.shape
#1.  Flow id
# 2. Dest Id 
# 3. Src id 
# 4. Src port
# 5. Dest port 
# 6. Time stamp 
# 7. Protocol type

(68424, 77)

In [7]:
df_test.drop(['Flow ID', 'Src IP' ,'Dst IP' ,'Timestamp','Src Port','Dst Port','Protocol'], inplace=True,axis = 1)
df_test.shape

(138722, 77)

In [8]:
df.head(5)

,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Fwd Pkt Len Std,Bwd Pkt Len Max,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,245230,44,40,124937.0,1071.0,9100,0,2839.477273,1839.508257,517,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Normal
1,1605449,107,149,1071.0,439537.0,517,0,10.009346,67.496680,27300,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Normal
2,53078,5,5,66.0,758.0,66,0,13.200000,29.516097,638,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Normal
3,6975,1,1,0.0,0.0,0,0,0.000000,0.000000,0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Normal
4,190141,13,16,780.0,11085.0,427,0,60.000000,130.042942,2596,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Normal


In [9]:
#Encoding categorical data i.e Label
pmap = {'Normal':0,'Probe ':1,'DDoS':1,'DoS':1,'BFA':1,'Web-Attack':1,'BOTNET':1,'U2R':1,'DDoS':1}
df_test['Label'] = df_test['Label'].map(pmap)


In [10]:
df_test.head(5)

,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Fwd Pkt Len Std,Bwd Pkt Len Max,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,4045,2,5,476,1803,476,0,238.0,336.582828,1803,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,284,1,1,0,0,0,0,0.0,0.000000,0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,4302,2,5,478,1803,478,0,239.0,337.997041,1803,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,83,1,1,0,0,0,0,0.0,0.000000,0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,3760,2,5,477,1803,477,0,238.5,337.289935,1803,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [11]:
df_test.Label.unique()

array([ 1., nan])

In [12]:
df_test.isna().sum()


Flow Duration          0
Tot Fwd Pkts           0
Tot Bwd Pkts           0
TotLen Fwd Pkts        0
TotLen Bwd Pkts        0
                   ...  
Idle Mean              0
Idle Std               0
Idle Max               0
Idle Min               0
Label              84785
Length: 77, dtype: int64

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix, accuracy_score

In [14]:
min_col = {}
max_col = {}
balance = {}
for col in df:
    max_col[col]= df[col].max()
    min_col[col]= df[col].min()
    #plt.hist(df[col])
    #plt.show()
    balance[col]= max_col[col] + min_col[col]

result = pd.DataFrame([min_col, max_col, balance], index=['min', 'max', 'balance'])
print(result)    

         Flow Duration  Tot Fwd Pkts  Tot Bwd Pkts  TotLen Fwd Pkts  \
min               -154             0             1              0.0   
max          119999993         16928         34094       31600000.0   
balance      119999839         16928         34095       31600000.0   

         TotLen Bwd Pkts  Fwd Pkt Len Max  Fwd Pkt Len Min  Fwd Pkt Len Mean  \
min                  0.0                0                0               0.0   
max          107000000.0            64239             3900           32119.5   
balance      107000000.0            64239             3900           32119.5   

         Fwd Pkt Len Std  Bwd Pkt Len Max  ...  Fwd Seg Size Min  Active Mean  \
min              0.00000                0  ...                 0          0.0   
max          45423.83252            64239  ...                 0  104000000.0   
balance      45423.83252            64239  ...                 0  104000000.0   

         Active Std   Active Max   Active Min    Idle Mean    Idle St

In [15]:
min_col = {}
max_col = {}
balance = {}
for col in df_test:
    max_col[col]= df_test[col].max()
    min_col[col]= df_test[col].min()
    #plt.hist(df[col])
    #plt.show()
    balance[col]= max_col[col] + min_col[col]

result = pd.DataFrame([min_col, max_col, balance], index=['min', 'max', 'balance'])
print(result)    

         Flow Duration  Tot Fwd Pkts  Tot Bwd Pkts  TotLen Fwd Pkts  \
min                  1             0             1                0   
max          119786058        910748          2039            77600   
balance      119786059        910748          2040            77600   

         TotLen Bwd Pkts  Fwd Pkt Len Max  Fwd Pkt Len Min  Fwd Pkt Len Mean  \
min                    0                0                0          0.000000   
max                80143             1448               44       1201.181818   
balance            80143             1448               44       1201.181818   

         Fwd Pkt Len Std  Bwd Pkt Len Max  ...  Fwd Seg Size Min  Active Mean  \
min              0.00000                0  ...                 0          0.0   
max            625.38812             2063  ...                 0   68100000.0   
balance        625.38812             2063  ...                 0   68100000.0   

          Active Std  Active Max  Active Min   Idle Mean    Idle Std 

In [16]:
df_test.shape

(138722, 77)

In [17]:
Y = df[['Label']]
X = df.drop(['Label',], axis=1)

sc = MinMaxScaler(feature_range = (0, 1))
X = sc.fit_transform(X)
print(X)

[[2.04486416e-03 2.59924386e-03 1.14392984e-03 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [1.33800086e-02 6.32088847e-03 4.34106708e-03 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [4.43599457e-04 2.95368620e-04 1.17326137e-04 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 ...
 [3.23749603e-05 5.90737240e-05 5.86630687e-05 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [1.72874788e-04 0.00000000e+00 2.93315343e-05 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [2.66083007e-05 5.90737240e-05 5.86630687e-05 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]]


In [18]:
print(X.shape)


(68424, 76)


In [19]:
test = np.array(X)

In [20]:
print(test)

[[2.04486416e-03 2.59924386e-03 1.14392984e-03 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [1.33800086e-02 6.32088847e-03 4.34106708e-03 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [4.43599457e-04 2.95368620e-04 1.17326137e-04 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 ...
 [3.23749603e-05 5.90737240e-05 5.86630687e-05 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [1.72874788e-04 0.00000000e+00 2.93315343e-05 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [2.66083007e-05 5.90737240e-05 5.86630687e-05 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]]


In [21]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.20, random_state=42)
print(X_train.shape, X_test.shape)
print(Y_train.shape, Y_test.shape)

(54739, 76) (13685, 76)
(54739, 1) (13685, 1)


In [22]:
Yy = df_test[['Label']]
Xx = df_test.drop(['Label',], axis=1)

sc = MinMaxScaler(feature_range = (0, 1))
Xx = sc.fit_transform(Xx)
print(Xx)

[[3.37601896e-05 2.19599714e-06 1.96270854e-03 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [2.36254542e-06 1.09799857e-06 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [3.59056814e-05 2.19599714e-06 1.96270854e-03 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 ...
 [3.43946541e-05 1.09799857e-06 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [4.91376054e-05 2.19599714e-06 1.96270854e-03 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [1.24388434e-06 1.09799857e-06 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]]


In [23]:
# X_train_out, X_test_out, Y_train_out, Y_test_out = train_test_split(Xx, Yy, test_size=0.40, random_state=42)
# print(X_train_out.shape, X_test_out.shape)
# print(Y_train_out.shape, Y_test_out.shape)

In [24]:
from sklearn import svm
clf = svm.OneClassSVM(nu=0.4, kernel="rbf", gamma=0.001)
clf.fit(X_train)

OneClassSVM(gamma=0.001, nu=0.4)

In [25]:
y_pred_train = clf.predict(X_train)
y_pred_test = clf.predict(X_test)


In [26]:
print(y_pred_train)
y_pred_train.shape

[ 1  1  1 ...  1 -1 -1]


(54739,)

In [27]:
print(y_pred_test)

[ 1 -1  1 ...  1  1 -1]


In [28]:
y_outliers = clf.predict(Xx)
y_outliers.shape


(138722,)

In [29]:
print(y_outliers)

[-1  1 -1 ...  1 -1  1]


In [30]:
n_error_train = y_pred_train[y_pred_train == -1].size
n_error_test = y_pred_test[y_pred_test == -1].size
n_error_outliers = y_outliers[y_outliers == 1].size

In [31]:
print(n_error_train)
print(n_error_test)
print(n_error_outliers)

21413
5370
5609


# For normal 

In [32]:
from sklearn.metrics import accuracy_score
y_true = np.ones((54739,1), dtype=np.int)
accuracy_score(y_true, y_pred_train)

0.6088163832002776

# For anomaly

In [33]:
from sklearn.metrics import accuracy_score
y_true = np.ones((138722,1), dtype=np.int)
accuracy_score(y_true, y_outliers)
1-accuracy_score(y_true, y_outliers)

0.9595666152448783

In [34]:
from sklearn.metrics import classification_report
print(classification_report(y_true,y_outliers))

              precision    recall  f1-score   support

          -1       0.00      0.00      0.00         0
           1       1.00      0.04      0.08    138722

    accuracy                           0.04    138722
   macro avg       0.50      0.02      0.04    138722
weighted avg       1.00      0.04      0.08    138722



/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
